In [ ]:
import cv2
from keras.models import load_model
import numpy as np
import os

model = load_model('emojinator.h5')

def main():
    emojis = get_emojis()
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

        x, y, w, h = 300, 50, 350, 350
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break

def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    
    background_mask = 255 - overlay_mask

    
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

keras_predict(model, np.zeros((50, 50, 1), dtype=np.uint8))
main()


Using TensorFlow backend.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
14 0.6365949
1 0.98268867
14 0.9104586
14 0.95770174
14 0.3854789
7 0.6011448
14 0.71976626
2 0.9045347
14 0.9999261
14 0.9942596
13 0.7092888
10 0.7268373
10 0.9608981
14 0.53666747
11 0.88551813
11 0.8440378
11 0.9784707
11 0.7049448
1 0.97348434
11 0.4552706
11 0.93129987
1 0.98505104
11 0.62102854
1 0.40405312
1 0.9980521
1 0.99958175
8 0.40175954
1 0.98067105
1 0.99961567
14 0.37281922
1 0.99402195
1 0.9460654
13 0.74099344
1 0.9987662
5 0.90539783
11 0.99763536
1 0.8724192
8 0.8045714
10 0.35414484
12 0.6363569
1 0.9999727
14 0.99966395
1 0.9861632
14 0.95544463
13 0.4784466
11 0.5538578
13 0.5997348
4 0.5389048
14 0.99859875
14 0.9999689
14 0.9940392
14 0.99892753
10 0.9610421
10 0.95565546
14 0.6214939
6 0.6349191
10 0.49068618
14 0.9643967
14 0.94565374
14 0.86517966
14 0.9854277
14 0.9901288
14 0.72728735
14 0.9998016
14 0.9073402
14 0.94283736
14 0.93167335
14 0.7515011
14 0.8920672
14 0.98353344
14 0.9932086
14 0.9998479
14 0.98641104
1

5 0.5274041
5 0.74412817
5 0.9641828
5 0.9706425
5 0.965036
5 0.98013914
14 0.86410546
13 0.93128693
13 0.9400773
13 0.9941327
13 0.8933012
13 0.49698725
7 0.76965016
7 0.8711786
7 0.9548929
7 0.6634641
7 0.7673218
7 0.6474603
11 0.5466
11 0.646439
7 0.662873
7 0.74514884
7 0.73558885
7 0.6680591
7 0.77494574
7 0.61538064
7 0.703012
7 0.59324396
7 0.7548241
7 0.627609
7 0.7881755
7 0.8111597
7 0.7469128
7 0.70777696
7 0.90847045
7 0.93174416
7 0.93801403
7 0.9562421
7 0.943717
7 0.9719315
7 0.9669271
7 0.9745232
7 0.9739363
7 0.9264215
7 0.96196026
7 0.94535303
7 0.9750283
7 0.95554924
7 0.9716365
7 0.9623246
7 0.9566911
7 0.96918184
7 0.97310334
7 0.97144586
7 0.97663957
7 0.97322637
7 0.9848379
7 0.9931591
7 0.9882732
7 0.980881
7 0.97545594
7 0.97709835
7 0.9816305
7 0.9866001
7 0.98359793
7 0.9806198
7 0.9572579
7 0.9674754
7 0.9776683
7 0.9811214
7 0.9648679
7 0.96706855
14 0.5721367
12 0.892217
12 0.7562518
12 0.95011055
12 0.95514035
12 0.9398531
12 0.92891574
12 0.9423289
12 0.